In [1]:
from genepy.utils import helper as h

from taigapy import TaigaClient
from celligner import Celligner
tc = TaigaClient()
from bokeh.plotting import output_notebook
from depmapomics import tracker as track
#autoreload
%load_ext autoreload
%autoreload 2
output_notebook()

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:14: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "l2_norm" failed type inference due to: No implementation of function Function(<function norm at 0x7fed882e1b70>) found for signature:
 
 >>> norm(x=array(float32, 2d, A), axis=Literal[int](1))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'norm_impl': File: numba/np/linalg.py: Line 2352.
    With argument(s): '(x=array(float32, 2d, A), axis=int64)':
   Rejected as the implementation raised a specific error:
     TypeError: norm_impl() got an unexpected keyword argument 'x'
  raise

/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:199: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "adjust_s_variance" failed type inference due to: NameError: name 'sq_dist_to_line' is not defined
  @jit(float32(float32[:, :], float32[:, :], float32[:], float32[:], float32), nogil=True)
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:199: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "adjust_s_variance" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", line 207:
def adjust_s_variance(data1, data2, curcell, curvect, sigma):
    <source elided>
    totalprob2 = 0.
    for samecell in data2:
    ^

  @jit(float32(float32[:, :], float32[:, :], float32[:], float32[:], float32), nogil=True)
/home/jeremie/miniconda3/lib/pytho

Loading BokehJS ...

In [5]:
# load from taiga public (figshare link)
# load internal expression,
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  # https://depmap.org/portal/download/
# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here:
TCGA_expression = tc.get(name='celligner-input-9827',
                         file='tumor_expression')

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [43]:
# subset gene names to ensembl ids only
CCLE_expression = CCLE_expression[CCLE_expression.columns[:-92]]
CCLE_expression.columns = list(map(lambda x: x.split(
    ' (')[1][:-1] if ' (' in x else x, CCLE_expression.columns))

common = set(CCLE_expression.columns).intersection(
    set(TCGA_expression.columns))
CCLE_expression = CCLE_expression[list(common)]
TCGA_expression = TCGA_expression[list(common)]

In [31]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY/edit#gid=746987867 
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually, found also here: 

No dataset version provided. Using version 1.


In [32]:
# transforming annotations
CCLE_annotation = CCLE_annotation.drop_duplicates('arxspan_id').set_index("arxspan_id")
CCLE_annotation = CCLE_annotation.loc[CCLE_expression.index, ["origin", 'subtype']].rename(columns={"origin": "tissue_type", "subtype": 'disease_type'})
CCLE_annotation["cell_type"] = "cancer cell line"

TCGA_annotation = TCGA_annotation.set_index("sampleID").loc[TCGA_expression.index,["lineage",
"subtype"]].rename({"lineage":"tissue_type", "subtype": 'disease_type'})
TCGA_annotation['cell_type'] = "tumor sample"

In [35]:
TCGA_expression.index TCGA_annotation.index

SyntaxError: invalid syntax (<ipython-input-35-d0d98637db54>, line 1)

In [39]:
my_alligner = Celligner(snn_kwargs={"neighbor_num": 30, "min_shared_neighbor_proportion": 1/15,}, make_plots=True)

my_alligner.fit(TCGA_expression.iloc[:800], TCGA_annotation.iloc[:800])

fetching gene names from biomart cache
using only usefull genes
looking at 800 samples.
found 33980 common genes
creating a fit dataset..
reducing dimensionality...
clustering...


doing differential expression analysis on the clusters
running differential expression on 7 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
done


In [50]:
my_alligner.common_genes

Index(['ENSG00000240041', 'ENSG00000206932', 'ENSG00000213700',
       'ENSG00000157045', 'ENSG00000235595', 'ENSG00000091732',
       'ENSG00000238001', 'ENSG00000153898', 'ENSG00000143630',
       'ENSG00000231795',
       ...
       'ENSG00000050405', 'ENSG00000161642', 'ENSG00000253026',
       'ENSG00000194717', 'ENSG00000241769', 'ENSG00000197461',
       'ENSG00000256171', 'ENSG00000255329', 'ENSG00000104517',
       'ENSG00000230448'],
      dtype='object', length=33980)

In [52]:
[0]*3 + [3]*1

[autoreload of celligner failed: Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jeremie/miniconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jeremie/miniconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/jeremie/cel

[0, 0, 0, 3]

In [57]:
np.vstack([my_alligner.fit_reduced,my_alligner.fit_reduced]).shape

(1600, 70)

In [ ]:
centered_fit_input.loc[self.fit_clusters==val] = self.fit_input.loc[self.fit_clusters==val]\
        - self.fit_input.loc[self.fit_clusters==val].mean(axis=0)

In [ ]:
my_alligner.transform(CCLE_expression.iloc[:400], CCLE_annotation.iloc[:400])

looking at 800 samples.
found 29593 common genes
creating a transform input..
reducing dimensionality...
clustering..


doing differential expression analysis on the clusters..
running differential expression on 6 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
there is 0.339 overlap between the fit and transform dataset in their most variable genes
doing cPCA..
transform
regressing out the cPCA components..
> /home/jeremie/celligner/celligner/__init__.py(354)transform()
-> transformed_fit = self.fit_input - LinearRegression(fit_intercept=False).fit(
(Pdb) l
349  	    # regress out the cPCA components from the data
350  	    print('regressing out the cPCA components..')
351  	    # take the residuals of the linear regression of fit_input with the cpca_loadings
352  	    del centered_transform_input, centered_fit_input
353  	    import pdb; pdb.set_trace()
354  ->	    transformed_fit = self.fit_input - LinearRegression(fit_intercept=False).fit(
355  	      cpca_loadings.T, self.transform_input.T).coef_.T
356  	    transformed_transform = self.tr

In [ ]:
my_alligner.save('temp/demo/')

In [ ]:
my_alligner.plot()